In [0]:
# https://pypi.python.org/pypi/pydot
!apt-get -qq install -y graphviz && pip install -q pydot

In [0]:
# データセット及びモジュールの取得
!git clone https://github.com/pphos/rock_paper_scissors.git

Cloning into 'rock_paper_scissors'...
remote: Enumerating objects: 298, done.
remote: Counting objects: 100% (298/298), done.
remote: Compressing objects: 100% (153/153), done.
remote: Total 298 (delta 168), reused 269 (delta 142), pack-reused 0
Receiving objects: 100% (298/298), 593.29 MiB | 31.71 MiB/s, done.
Resolving deltas: 100% (168/168), done.
Checking out files: 100% (31/31), done.


In [0]:
# モジュールの移動
!mv './rock_paper_scissors/src/modules' ./

In [0]:
# データセット結合
!cat ./rock_paper_scissors/datasets/rock_paper_scissors.tar.gz.* > datasets.tar.gz

In [0]:
# データセットの解凍
!tar -zxvf datasets.tar.gz
!rm datasets.tar.gz

splited_dataset/
splited_dataset/eval_features/
splited_dataset/eval_features/data.npy
splited_dataset/eval_features/target_label.pkl
splited_dataset/eval_features/target.npy
splited_dataset/training_features/
splited_dataset/training_features/data.npy
splited_dataset/training_features/target_label.pkl
splited_dataset/training_features/target.npy


In [0]:
import os
from modules.utils.preprocess import load_and_preprocess_dataset
from modules.models.model_selection import select_model
from modules.utils.io import init_result_save_dir

Using TensorFlow backend.


In [0]:
def init_each_config(target_label, save_root, unique=False):
  """
  各種コンフィグの初期化
  # Arguments
    target_label : 正解ラベル (pickle形式)
    save_root    : 結果を保存するディレクトリのパス
    unique       : 保存ディレクトリに一意の名前をつけるかのフラグ
  # Returns
    model_conf   : モデル情報を格納した辞書
     Attribute)
      name                  : モデル名
      save_dir              : モデルの保存ディレクトリパス
      epochs                : モデルを訓練するエポック数 (int)
      batch_size            : 勾配の更新を行うバッチサイズ (int)
      validation_split      : 訓練データの中で検証データとして使う割合 (float)
      loss                  : 損失関数
      optimizer             : 最適化関数
      metrics               : 評価やテストの際にモデルを評価するための評価関数のリスト
      set_class_weight      : クラスごとに異なる重みづけを行うかのフラグ (bool) 
                             (各カテゴリの不均衡の解消用)
      enable_early_stopping : 学習の早期終了を行うかのフラグ (bool)
      
    save_conf    : 結果の保存に関する情報を格納した辞書
     Attribute)
      digits                : 出力結果の有効数字の桁数 (int)
      cnf_matrix_title      : Confusion Matrixの図につけるタイトル
      cnf_matrix_fname      : Confusion Matrixの保存名 (画像ファイル)
      text_fname            : classification_reportの保存名 (テキストファイル)
      save_prefix           : 結果ファイルに付与するプレフィックス
  """
  # 保存対象ディレクトリの初期化
  model_name = 'MnistCNN'
  save_dirs = init_result_save_dir(name=model_name,
                                   save_root=save_root,
                                   unique=unique)
  
  # model_conf の初期化
  model_conf = {
      'name': model_name,
      'save_dir': save_dirs['model'],
      'epochs': 1,
      'batch_size': 32,
      'validation_split': 0.1,
      'loss': 'categorical_crossentropy',
      'optimizer': 'Adam',
      'metrics': ['accuracy'],
      'set_class_weight': False,
      'enable_early_stopping': False
  }
  
  # save_conf の初期化
  save_conf = {
      'save_dir': save_dirs['figure'],
      'digits': 4,
      'target_names': target_label,
      'cnf_matrix_title': None,
      'cnf_matrix_fname': 'confusion_matrix.png',
      'text_fname': 'classification_report.txt',
      'save_prefix': None
  }
  
  return model_conf, save_conf

In [0]:
# データセットのパスの設定
train_root = './splited_dataset/training_features'
test_root = './splited_dataset/eval_features'

base_file_names = ['data.npy', 'target.npy', 'target_label.pkl']
train_paths, test_paths = {}, {}
for value in base_file_names:
  key = os.path.splitext(value)[0]
  train_paths[key] = os.path.join(train_root, value)
  test_paths[key] = os.path.join(test_root, value)
  

In [0]:
# データセットの読み込み兼前処理
train_dataset = load_and_preprocess_dataset(train_paths['data'],
                                            train_paths['target'],
                                            train_paths['target_label'])
test_dataset = load_and_preprocess_dataset(test_paths['data'],
                                           test_paths['target'],
                                           test_paths['target_label'])

In [0]:
# 各種コンフィグの初期化
save_root = './results'
model_conf, save_conf = init_each_config(train_dataset.target_label, save_root, unique=False)

In [0]:
# 訓練モデルの選択
model = select_model(model_conf['name'])
model = model(name=model_conf['name'],
              input_shape=train_dataset.X.shape[1:],
              nb_classes=len(train_dataset.target_label),
              save_dir=model_conf['save_dir'])

In [0]:
# 訓練モデルのサマリーの表示
model.model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 254, 254, 8)       224       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 252, 252, 16)      1168      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 126, 126, 16)      0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 126, 126, 16)      0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 254016)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 32)                8128544   
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
__________

In [0]:
# モデルの訓練
history = model.train(train_dataset.X, train_dataset.y, model_conf,)

Train on 2025 samples, validate on 225 samples
Epoch 1/1
2025/2025 [==============================] - 10s 5ms/step - loss: 1.0528 - acc: 0.6504 - val_loss: 0.2893 - val_acc: 1.0000

Epoch 00001: val_loss improved from inf to 0.28935, saving model to ./results/MnistCNN/model/MnistCNN_weights.001-1.0528.hdf5


In [0]:
# モデルの評価
result = model.evaluate(test_dataset.X, test_dataset.y)

750/750 [==============================] - 1s 1ms/step


In [0]:
# モデルの評価結果の保存
model.visuaize_result(save_conf)

In [0]:
# 結果のダウンロード
import shutil
from google import colab

shutil.make_archive(save_root, 'zip', save_root)

zip_file_name = "{}.zip".format(save_root)
colab.files.download(zip_file_name)